In [1]:
import numpy as np
import time

In [2]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [3]:
rootNode = treeNode('pyramid',9,None)
rootNode.children['eye'] = treeNode('eye',13,None)
rootNode.children['phoenix'] = treeNode('phoenix',3,None)
rootNode.disp()

   pyramid   9
     eye   13
     phoenix   3


In [4]:
def updateHeader_origin(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        #print("move node", nodeToTest.count)
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [5]:
# 上面的 updateHeader_origin 是树上原始的附属代码
# 感觉这个遍历 没有必要，可以直接插在链表的开头，这个应该没有区别
def updateHeader_new(nodeToTest, targetNode):
    #if nodeToTest != None :
    targetNode.nodeLink = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [6]:
def printHeader(ht):
    for head in ht:
        print(head, ':', end='')
        data = ht[head]
        print(data[0], end='')
        nl = data[1]
        while nl != None :
            print(' --> {', end ='')
            pn = nl
            while pn != None:
                if(pn.name != 'Null Set'):
                    print(pn.name, end ='')
                pn = pn.parent
            print('}',nl.count, end ='')
            nl = nl.nodeLink
        print('')

In [7]:
def updateTree(items, inTree, headerTable, count, updateHeader):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1: #call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count, updateHeader)

In [8]:
def createTree(dataSet, minSup=1, updateHeader= updateHeader_new): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable.keys()):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable :
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            # 排序 
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count, updateHeader)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

In [9]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [10]:
simpDat = loadSimpDat()

In [11]:
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [12]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [13]:
dataSet = createInitSet(simpDat)

In [14]:
retTree, headerTable = createTree(dataSet, 3, updateHeader_new)

In [15]:
retTree.disp()
printHeader(headerTable)

   Null Set   1
     z   5
       r   1
       x   3
         y   2
           s   2
             t   2
         r   1
           y   1
             t   1
     x   1
       r   1
         s   1
r :3 --> {rz} 1 --> {rxz} 1 --> {rx} 1
z :5 --> {z} 5
x :4 --> {xz} 3 --> {x} 1
y :3 --> {yxz} 2 --> {yrxz} 1
s :3 --> {syxz} 2 --> {srx} 1
t :3 --> {tsyxz} 2 --> {tyrxz} 1


In [16]:
# 这个递归没有必要
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [17]:
def ascendTree_new(leafNode): #ascends from leaf node to root
    prefixPath = []
    ln = leafNode
    while ln.parent != None:
        prefixPath.append(ln.name)
        ln = ln.parent
    return prefixPath

In [18]:
# basePat 这个参数好像没有用
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [19]:
def findPrefixPath_new(treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = ascendTree_new(treeNode)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [20]:
findPrefixPath('t',headerTable['t'][1])

{frozenset({'s', 'x', 'y', 'z'}): 2, frozenset({'r', 'x', 'y', 'z'}): 1}

In [21]:
findPrefixPath_new(headerTable['t'][1])

{frozenset({'s', 'x', 'y', 'z'}): 2, frozenset({'r', 'x', 'y', 'z'}): 1}

In [22]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath_new(headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print ('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)            

In [23]:
# 结果和树上
freqItems = []
mineTree(retTree, headerTable, 3, set([]), freqItems)

conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'y', 'x'}
   Null Set   1
     z   3
conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'t'}
   Null Set   1
     y   3
       z   3
         x   3
conditional tree for:  {'t', 'z'}
   Null Set   1
     y   3
conditional tree for:  {'t', 'x'}
   Null Set   1
     y   3
       z   3
conditional tree for:  {'t', 'z', 'x'}
   Null Set   1
     y   3
conditional tree for:  {'x'}
   Null Set   1
     z   3


In [24]:
freqItems

[{'r'},
 {'y'},
 {'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]

In [25]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]

In [26]:
initSet1 = createInitSet(parsedDat)

In [27]:
time1 = time.time()
myFPtree1,myHeaderTab1 = createTree(initSet1,100000) #最小频数为100000
time2 = time.time()
print('creating the fp-tree cost ',time2-time1,'s')

creating the fp-tree cost  3.7633955478668213 s


In [28]:
time3 = time.time()
myFreqList = []
mineTree(myFPtree1,myHeaderTab1,100000,set([]),myFreqList)
time4 = time.time()
print (len(myFreqList))
print('creating the fp-tree cost ',time4-time3,'s')

conditional tree for:  {'1'}
   Null Set   1
     6   107404
conditional tree for:  {'3'}
   Null Set   1
     11   9718
     6   186289
       11   117401
conditional tree for:  {'11', '3'}
   Null Set   1
     6   117401
conditional tree for:  {'11'}
   Null Set   1
     6   261773
9
creating the fp-tree cost  0.002527475357055664 s


In [29]:
myFreqList

[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]